# Manual operations

Subsections should be self-contained

## Manual filter of datasets

In [ ]:
from pinn.io import load_ds, write_tfrecord
def filter_ef(datum):
    import tensorflow as tf
    return (datum['e_data']<-2944.) and (tf.reduce_max(tf.abs(datum['f_data']))<0.2)
ds = load_ds('../pils-v3.yml').filter(filter_ef)

## Generation of geometry from experiments

In [ ]:
from ase.io import read, write
from glob import glob
import os
for comp in ['a0b0i32', 'a32b32i0', 'a16b16i16']:
    for rho in ['1.16', '1.08']:
        geo = read(f'../exp/trial-adam/prod/gen31/nvt-340k-5ns-0/{comp}-r{rho}/asemd.traj', index='-1')
        write(f'../skel/init/trial-geo/{comp}-r{rho}.xyz', geo)
        os.mkdir(f'fake/{comp}-r{rho}/')
        write(f'fake/{comp}-r{rho}/asemd.traj', [geo])

## Writing out trajectories for visualization

In [ ]:
def smart_wrap(atoms):
      #!/usr/bin/env python
    import numpy as np
    from ase import Atoms, neighborlist
    from ase.io import write
    from scipy import sparse
    natoms =  len(atoms)
    
    assert (atoms.cell.angles() == 90.).all(), "Only orthogonal cells allowed."
    cell = atoms.cell.diagonal()
    atoms.wrap()
  
    cutoff = {
        ("H", "C"): 1.74,
        ("H", "N"): 1.9,
        ("H", "O"): 1.9,
        ("C", "C"): 2,
        ("C", "N"): 2,
        ("C", "O"): 2,
    }

    nl_i, nl_j, nl_d = neighborlist.neighbor_list("ijd", atoms, cutoff, self_interaction=False)
    conMat = sparse.dok_matrix((natoms, natoms), dtype=np.int8)
    conMat[nl_i, nl_j] = 1  # we have several running indices here prefixed by (nl, mol, h)
    conMat[nl_j, nl_i] = 1  # v---- shamelessly taken from the ase documentation
    n_mol, mol_assign = sparse.csgraph.connected_components(conMat)
    
    for mol_i in range(n_mol):
        pos_mol = atoms.positions[mol_assign==mol_i]
        mass_mol = atoms.get_masses()[mol_assign==mol_i]
        pos_mol -= np.rint((pos_mol-pos_mol[:1,:])/cell[None,:])*cell[None,:]
        com_mol = mass_mol@pos_mol/(mass_mol.sum())
        pos_mol -= np.rint(com_mol/cell[None,:]-0.5)*cell[None,:]
        atoms.positions[mol_assign==mol_i] = pos_mol
        
def unwrap(atoms_prev, atoms_next):
    cell = atoms_next.cell.diagonal()
    atoms_next.positions -= np.rint((atoms_next.positions-atoms_prev.positions)
                                    /cell[None,:])*cell[None,:]    

In [ ]:
from tips.io import load_ds
from ase.io import write

traj = load_ds('../exp/adam1-sin-run1/emd/gen28/m16i16-r1.08/asemd.traj',fmt='asetraj', index='::100')
traj_ase = traj.convert(fmt='ase')

for idx, atoms in enumerate(traj_ase):
    if 'stress' in atoms.calc.results : del atoms.calc.results['stress']    
    if  idx%100==0:
        print(f'\r{idx}', end='')
        smart_wrap(atoms)
    else:
        unwrap(traj_ase[idx-1], traj_ase[idx])
write('tmp.xyz', traj_ase)

In [ ]:
from tips.io import load_ds
from ase.io import write

traj = load_ds('../exp/adam1-sin-run1/merge/gen28/m16i16-r1.16/merged.traj',fmt='asetraj', index=':')
traj_ase = traj.convert(fmt='ase')
for idx, atoms in enumerate(traj_ase):
    if 'stress' in atoms.calc.results : del atoms.calc.results['stress']    
    if  True:
        print(f'\r{idx}', end='')
        smart_wrap(atoms)
    else:
        unwrap(traj_ase[idx-1], traj_ase[idx])
write('tmp.xyz', traj_ase)